# Welcome to __PERFUMEme.py__ 🧪👃🧴⚛️⚗️

## Introduction
Perfumes are a part of our everyday lives—whether it’s a subtle floral hint or a bold, spicy statement, most people have a signature scent that others come to recognize them by. But have you ever wondered what’s actually in your favorite fragrance? Perfumes are complex mixtures of countless molecules, and each one plays a unique role in shaping the scent you love.

So how can you figure out which molecule is responsible for that special note in your perfume? And once you have a molecule in mind, how do you know if it’s safe, aromatic, or even used in other fragrances?

Meet __PERFUMEme.py__, — a Python package that helps demystify the chemistry behind your scent. Simply input a molecule, and the tool will tell you:

- Whether the molecule is fragrant or toxic
- Its key physical properties, such as boiling point
- How it evaporates over time, via an evaporation model
- And best of all, which perfumes (when data allows) contain that exact molecule

Unlock the secrets behind the scents — one molecule at a time.

The main chemistry involved is organic chemistry and physical chemistry, particularly focusing on volatility and evaporation.


## 🛠️ Project Overview

The __PERFUMEme__ project is a Python-based toolkit designed to bridge the gap between perfumery and cheminformatics by enabling structured, molecule-level exploration of commercial fragrances. Its primary function is to understand if a certain molecule is usable in the perfume industry and understand why or why not.

Perfumes are typically described by their olfactory notes (e.g., floral, woody, spicy) and ingredient names, but rarely include explicit chemical representations. This lack of molecular data limits the potential for computational analysis and machine learning applications in fragrance design. PERFUMEme addresses this issue by enabling users to convert perfume data—including name, brand, ingredient names, and scent notes—into a format enriched with chemical identifiers, suitable for further computational analysis, clustering, or visualization.

The package processes structured JSON input and performs the following key tasks:

- Queries PubChem using molecule names to retrieve SMILES strings
- Handles ambiguous or unrecognized molecule names with graceful fallbacks.
- Returns structured data that can be visualized and analyzed.

By combining publicly available molecular data with perfume metadata, PERFUMEme provides a foundation for exploring relationships between molecular composition and fragrance perception, opening the door to computational fragrance classification, similarity scoring, and even automated formulation tools. The typical user of this package would be fragrance chemists, cosmetic scientists, cheminformatics researchers, or curious enthusiasts with a background in organic or physical chemistry — especially those interested in functional groups and molecular volatility. 

_This may be a bit too repetitive._

## 🧰 Material and methods

### 🗂 Data Sources
Molecular data were retrieved from PubChem, a publicly accessible chemical database maintained by the National Center for Biotechnology Information (NCBI). The PubChem PUG REST API was used to access compound properties, such as their molecular names, SMILES strings, boiling point, enthalpy of vaporisation and pressure value.

The fragrance data, including perfume names, brands, notes, and associated molecules, were compiled manually and stored in a local JSON file. This file served as the data set for the perfume_molecule.py functions.

### 𝌤 Package Structure
The __PERFUMEme__ package was implemented in Python and organized into several functional modules:

- scraper.py: includes functions used to add the molecules listed in the perfumes in perfume.json to a new database and on to addtheir smiles from PubChem, used withodors.csv to add their respective odors (if they had smell).
- utils.py: contains utility functions for validating molecule data, handling API errors, and processing JSON inputs.
- data/: directory containing the JSON files with structured perfume and molecular data.

The package was developed to be modular, allowing users to integrate new data sources or extend the current capabilities without modifying core functions. For example, adding a molecule with its odor and SMILES to molecules.json.

## 📊 Results and discussions


### Functions in __utils.py__

##### 🧪 SMILES Extraction

The SMILES strings for each molecule are retrievable using the __get_smiles__ function. A casual user might not know the SMILES of the molecule they want to analyse, so this function makes it easier for them. It is also used in the __resolve_input_to_smiles_and_cid__ function. A simple example usage is presented below.

In [2]:
import os
import sys
sys.path.append(os.path.abspath("../src"))


from perfumeme.utils import get_smiles
citronellol_smiles= get_smiles("Citronellol")
print (citronellol_smiles)

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

This function can handle API queries, error checking, and returns the results in a structured format.


##### Resolving the input

The function __resolve_input_to_smiles_and_cid__, used in the main functions, is what allows the user to input either a SMILES or the compound name. This is a function to make the main functions easier to use and accessable. It also uses the __get_cid_from_smiles__ function, which retrieves the PubChem Compound ID (CID) corresponding to a given SMILES string.

When developping the __main_functions__, the distinction between a molecule's name and its SMILES was done by looking at the first letter of the input. This raised a problem when the name of the molecule sarted with a "C" and if the SMILES didn't start with a "C" (like water for example). There is now a "try-first, fallback-on-exception" pattern to detect if the input is a SMILES or a molecule name.

##### Get the odor

This function is used in __scraper.py__ to get the odors of a molecule and add them to the database.

##### Getting information from PubChem

The function __get_pubchem_description__ is used in the function __has_a_smell__ to get the descriptive information associated with a compound identified by its CID, such as general chemical information and properties. This description corresponds to the table presented at the top of the PubChem page. For example, this is what you can find on PubChem for the citronellol molecule, compared to the result of the function. Thanks to the example above, the SMILES for citronellol is known and can be used for this function. 

In [2]:
from perfumeme.utils import get_pubchem_description, get_cid_from_smiles
from IPython.display import display, HTML

citronellol_cid = get_cid_from_smiles(citronellol_smiles)
print (citronellol_cid)

print("From PubChem website:")
screenshot_html = '<img width="400" alt="PubChem description of citronellol" src="https://github.com/mlacrx/perfumeme/blob/main/assets/citronellol_pubchem.png?raw=true">'
display(HTML(screenshot_html))
print(f"from function {get_pubchem_description(citronellol_cid)}")

### RUN

ModuleNotFoundError: No module named 'perfumeme'

The function __get_pubchem_record_sections__ gets the structured data sections for a compound from PubChem using its CID. It is used in several functions of the __main_functions.py__. It allows the functions to find the need information in the right section of the JSON.

These functions are essential for the successfull operation of the function.

### Function in __main_functions.py__

One of the main purpouses of this package is to determine if a given molecule is usable in a perfume. Multiple factors need to be taken into account to determine this. Such as if the molecule has a smell, is too toxic for the skin and see if it will stay long enough.

#### 🌸 Does it have a smell?

The function __has_a_smell__ verifies if a given molecule is odorous. Thanks to the function __resolve_input_to_smiles_and_cid__, either the SMILES or the name of the molecule can be introduced in the input. This function returns __True__ if it does, and __False__ if it doesn't. But of course even without a smell, some molecules are present in perfumes.

In [41]:
from perfumeme.main_functions import has_a_smell

print (has_a_smell("Citronellol"))

False


#### ☠️ Is it toxic?

The function __is_toxic_skin__ determines with a bool, using the PubChem safety data, if a given compound is toxic on the skin. Even if it returns __False__, it doesn't rule out its use in a perfume, as sometimes it can still be used in a small concentration.

#### 📈 Evaporation trace

The function __evaporation_trace__ accomplishes multiple tasks. Its main goal is to plot the evaporation trace of the molecule using thermodynamic data from PubChem. This graph show the relative concentration as a function of time. This will then be used, in the function __usable_in_perfume__ to determine if the molecule is a base note, a heart note or a top note, based on the vapor pressure or boiling point. For this reason, __evaporation_trace__ also returns the vapor pressure at a certain temperature, the boiling point and the enthalpy of vaporisation of the entered molecule. Below is an example for citronellol.

In [ ]:
from perfumeme.main_functions import evaporation_trace

molecule = "citronellol"

vapor_pressure, boiling_point, vp_temp, enthalpy, image_path = evaporation_trace(molecule)

print(f"💨 Vapor Pressure: {vapor_pressure} mmHg")
print(f"🔥 Boiling Point: {boiling_point} °C")
print(f"🌡️ Vapor Pressure Measured at: {vp_temp} °C")
print(f"⚡ Enthalpy of Vaporization: {enthalpy} J/mol")

The function encountered a significant challenge due to the lack of standardization on PubChem’s web pages. Unlike other databases where the structure and presentation of information are consistent across all records, PubChem's molecule pages vary significantly depending on the compound. This inconsistency posed a problem when attempting to extract specific information, as the function needed to adapt to different page layouts and data formats. After numerous attempts with a variety of molecules, each displaying its data in unique ways, it became evident that a more flexible approach was necessary. Consequently, the function was restructured to handle these discrepancies, incorporating logic to process the different ways in which PubChem presents its information. This modification allowed the function to accept all possible formats and return the correct data, regardless of the molecule's page structure.

#### 🤔 So... can we use this molecule?

The function __usable_in_perfume__ evaluates whether a molecule is suitable for use in perfume formulations. It evaluates three key criteria to assess a molecule's suitability for perfumery: whether it has a detectable odor, whether it is safe for skin contact, and whether its volatility aligns with fragrance application. To classify the molecule as a top, heart, or base note, it primarily uses vapor pressure data, falling back on boiling point when necessary. Additionally, the function generates an annotated evaporation curve that visually highlights the corresponding note classification. An example for citronellol is given below.

In [ ]:
from perfumeme.usable_in_perfume import usable_in_perfume
molecule = "citronellol"
summary, plot = usable_in_perfume(molecule)
print(summary)
print(plot)

Initially, the assumption was made that for a molecule to be used in perfumery, it must possess a detectable odor and be safe for dermal exposure. As a result, the early version of the function automatically excluded any compound that was either odorless or flagged as potentially toxic. However, this approach proved to be too restrictive. In practice, the fragrance industry sometimes incorporates molecules that serve functional or technical roles—such as fixatives, stabilizers, or solvents—even if they don't contribute directly to the scent profile or have some level of toxicity under specific conditions. Recognizing this, the logic of the function was revised to allow for a more flexible and realistic evaluation, acknowledging that the presence of an odor or complete safety is not always a prerequisite for a molecule's use in a perfume formulation. Indeed, for certain molecules, if they are diluted enough, they don't have the same same safety restrictions.

### 👃 What odor does it give in the perfume?

The function __odor_molecule_perfume__ answers this question. It will return a dictionnary which matches the given molecule to a selection of perfumes in which it is present, and which smell it gives the perfume. Two functions were defined for it, or to be used seperately if you just wish to know the scent of the molecule for example. These two functions are __match_molecule_to_perfumes__ and __match_mol_to_odor__.

First, the database __perfumes.json__ was manually created with 29 perfumes. This list is limited as there is no database with all the perfumes and their ingredients. There is also the fact that not all perfume makers provide the list of molecules used in their perfumes. A separate database, __molecules.json__, contains all the molecules that are contained in the perfumes. To each is associated their SMILES and their odor. The odor was found from an additional database found online, __withodors.csv__. This last database contains a lot of unnecessary molecules for this package, which justifies the choice of creating a smaller database was made. The functions used to create these databases are found in __scraper.py__. It also doesn't include any molecules which are odorlesss.

#### 🤝 Match molecule to perfume

The function __match_molecule_to_perfumes__ will return a list of the perfumes which contain this molecule. This function is not case sensitive. Below is an example usage.

In [ ]:
from perfumeme.perfume_molecule import match_molecule_to_perfumes

# Find all perfumes containing Citronellol
citrionellol_perfumes = match_molecule_to_perfumes("citronellol")
print(citrionellol_perfumes)

#### 🧪 Match molecule to odor

The function __match_mol_to_odor__ returns a list which contains the odors of a given molecule. An example usage for this function:

In [ ]:
from perfumeme.perfume_molecule import match_mol_to_odor

# Find all odors of citronellol
citronellol_odors = match_mol_to_odor("citronellol")
print(citronellol_odors)

### 😥 Limitations of the module

The model classifies compounds into top, heart, or base notes primarily based on vapor pressure or boiling point. This approach doesn't account for the complex interactions and perceptions involved in fragrance development, potentially oversimplifying the classification.

The focus is on individual molecules without considering their behavior in mixtures, which is essential in perfumery. The synergistic effects and stability of compounds in a blend are not addressed.

The limitation for __odor_molecule_perfume__ is that there are only 29 perfumes listed in the database. Perfumes can always be added to it, but only manually. The molecules present in the newly added perfumes will then be automatically added to the __molecules.json__ database. The function __match_mol_to_odor__ only returns the odors of a molecule if it's in the database __molecules.json__. It is definetly possible to allow the user to add the molecule they insert to the database, if it's not yet present. This is an update that can be done in the future. However, __molecules.json__ contains all the molecule from __perfumes.json__, so if a user inputs a molecule which isn't in __molecules.json__ and so adds it to the databse, it won't be detected in a perfume.

## 👉 Conclusion

The PERFUMEme package provides an accessible and practical tool for evaluating the suitability of chemical compounds for use in perfumery. By integrating data from PubChem and applying criteria such as odor presence, dermal toxicity, and volatility, the package enables preliminary classification of molecules into fragrance note types (top, heart, or base). It also supports visual exploration through evaporation curves and simplifies molecular data extraction for further analysis. Furthermore, it presents, for a limited amount of perfumes, in which ones a molecule is used and its specified odor.

Throughout its development, key challenges such as inconsistent data structures on PubChem and rigid initial assumptions about odor and safety were successfully addressed through flexible code design and exception handling strategies. While the tool presents a valuable foundation for molecule screening and educational purposes, it is important to acknowledge its limitations—particularly the reliance on third-party data, simplified volatility modeling, the lack of mixture analysis, and the restrictions in the small database of perfumes.

To go further, one could incorporate functions and data to take into consideration the blend of molecules, to be able to give even more precise information on the molecules and their effect in a perfume.

In conclusion, PERFUMEme offers a thoughtful blend of chemical informatics and perfumery insight. It is best viewed as a helpful exploratory tool that complements, expert formulation knowledge and experimental validation in the fragrance development process.